### 1. Business Problem Understanding

Here we have predict the price of the house based on the facatlites of that the house is having

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('train.csv')
data

### 2. Data Understanding

In [ ]:
data.info()

In [ ]:
dict_types = dict(data.dtypes)
dict_types
continous = []
count = []
categorical = []
for name,type in dict_types.items():
    if type == str('float64'):
        continous.append(name)
    elif type == str('int64'):
        count.append(name)
    else:
        categorical.append(name)
print('The Continous variables are:  ',continous)
print('\nThe Count variables are:  ',count)
print('\nThe Categorical variables are:  ',categorical)
print(len(continous),len(count),len(categorical))


In [ ]:
numerical_data = data.drop(categorical,axis =1)
numerical_data

### 3. Data Preprocessing

In [ ]:
numerical_data.isnull().sum()

In [ ]:
data[categorical].isnull().sum()

In [ ]:
numerical_data.drop('Id',axis =1,inplace = True)
numerical_data.corr()

In [ ]:
data['Fence'].unique()

In [ ]:
data.drop('Fence',axis = 1 ,inplace = True)
data.drop('PoolQC',axis = 1 ,inplace = True)
data.drop('MiscFeature',axis = 1 ,inplace = True)
data.drop('Alley',axis = 1 ,inplace = True)
data.drop('Id',axis =1,inplace = True)

In [ ]:
for i in numerical_data:
    print('The variable name is:',i)
    sns.boxplot(data[i])
    plt.show()

In [ ]:
null = []
for i in numerical_data:
    st = data[i].isnull().sum()
    if(st > 0):
        null.append(i)
null

In [ ]:
data.isnull().sum()

In [ ]:
data['LotFrontage'].fillna(data['LotFrontage'].mean(),inplace = True)
data['MasVnrArea'].fillna(data['MasVnrArea'].mean(),inplace = True)
data['GarageYrBlt'].fillna(data['GarageYrBlt'].mean(),inplace = True)

In [ ]:
data['BsmtQual'].fillna(data['BsmtQual'].mode()[0],inplace = True)    
data['BsmtCond'].fillna(data['BsmtCond'].mode()[0],inplace = True) 
data['BsmtExposure'].fillna(data['BsmtExposure'].mode()[0],inplace = True)
data['BsmtFinType1'].fillna(data['BsmtFinType1'].mode()[0],inplace = True)
data['BsmtFinType2'].fillna(data['BsmtFinType2'].mode()[0],inplace = True)
data['MasVnrType'].fillna(data['MasVnrType'].mode()[0],inplace = True)
data['GarageType'].fillna(data['GarageType'].mode()[0],inplace = True)
data['GarageFinish'].fillna(data['GarageFinish'].mode()[0],inplace = True)       
data['GarageQual'].fillna(data['GarageQual'].mode()[0],inplace = True)       
data['GarageCond'].fillna(data['GarageCond'].mode()[0],inplace = True)
data['Electrical'].fillna(data['Electrical'].mode()[0],inplace = True)
data['FireplaceQu'].fillna(data['FireplaceQu'].mode()[0],inplace = True)

In [ ]:
dict_types = dict(data.dtypes)
dict_types
continous = []
count = []
categorical = []
for name,type in dict_types.items():
    if type == str('float64'):
        continous.append(name)
    elif type == str('int64'):
        count.append(name)
    else:
        categorical.append(name)
print('The Continous variables are:  ',continous)
print('\nThe Count variables are:  ',count)
print('\nThe Categorical variables are:  ',categorical)
print(len(continous),len(count),len(categorical))



##### Encoding the data

In [ ]:
"""X = pd.get_dummies(data,drop_first = True)
X"""

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(drop='first')
cat = enc.fit_transform(data[categorical])
data.drop(categorical,axis=1,inplace = True)
cat_df = pd.DataFrame(cat.toarray(), columns=enc.get_feature_names_out(categorical))
data = pd.concat([data, cat_df], axis='columns')


In [ ]:
X = data.drop('SalePrice',axis =1)
y = data['SalePrice']

In [ ]:
X

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size = 0.8,random_state = 12)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_model = StandardScaler()
X_train= sc_model.fit_transform(X_train)
X_test = sc_model.transform(X_test)

In [ ]:
from sklearn.decomposition import PCA
pca_model = PCA(n_components=0.97)
X_train_pca = pca_model.fit_transform(X_train)
X_test_pca = pca_model.transform(X_test)

In [ ]:
print(X_train.shape,X_train.shape)

In [ ]:
# Modelling
from sklearn.linear_model import LinearRegression
lr_model = LinearRegression()
lr_model.fit(X_train_pca,y_train)

# Predicting
ypred_train = lr_model.predict(X_train_pca)
ypred_test = lr_model.predict(X_test_pca)

# Evaluation
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
print("Rooted Mean Squared Error:",np.sqrt(mean_squared_error(y_test,ypred_test)))
print("Train Score:",r2_score(y_train,ypred_train))
print("Test Score:",r2_score(y_test,ypred_test))
##print("Cross Validation Score:",cross_val_score(lr_model,X,y,cv = 5))

##### Checking The Cheeklist for the Multiple Linear Regression Model

#### 1. Linearity of Errors

In [ ]:
total_res = y_test - ypred_test

In [ ]:
plt.scatter(y_test,total_res)
plt.xlabel('Observed variable')
plt.ylabel('Fitted Variable')
plt.show()

In [ ]:
sns.displot(total_res,kde = True,bins = 15)

In [ ]:
plt.scatter(ypred_test,total_res,c = 'g')
plt.axhline(c='blue')
#plt.axvline(x=15000,c='black')
plt.xlabel('fittedvalues')
plt.ylabel('resduals')
plt.show()


In [ ]:
import statsmodels.formula.api as smf
model1 = smf.ols('y~X',data = data).fit()
model1.summary()


#### 2. Polynomial Regression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

r2_train = []
r2_test = []

for i in range(1,3):
    polynomial_conversition = PolynomialFeatures(degree = i,include_bias = False)
    x_trainpoly = pd.DataFrame(polynomial_conversition.fit_transform(X_train_pca))
    x_testploy = pd.DataFrame(polynomial_conversition.fit_transform(X_test_pca))

    model = LinearRegression()
    model.fit(x_trainpoly,y_train)

    ypred = model.predict(x_testploy)
   

    r2_train.append(model.score(x_trainpoly,y_train))
    r2_test.append(model.score(x_testploy,y_test))

    print(i,r2_train,'\n')
    print(i,r2_test,'\n')
    



#### 2. Polynomial Regression

In [ ]:
# Polynomial Regression after we get the degree

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Assuming X_train_pca and X_test_pca are your PCA transformed data
polynomial_conversion = PolynomialFeatures(degree=1, include_bias=False)
X_train_polynomial = polynomial_conversion.fit_transform(X_train_pca)
X_test_polynomial = polynomial_conversion.transform(X_test_pca)  # Use transform instead of fit_transform for test data

poly_model = LinearRegression()
poly_model.fit(X_train_polynomial, y_train)

ypred_train = poly_model.predict(X_train_polynomial)
ypred_test = poly_model.predict(X_test_polynomial)

print("Train Score:", r2_score(y_train, ypred_train))
print('Test Score:', r2_score(y_test, ypred_test))

#### 3. Lasso Regulization

In [ ]:
from sklearn.linear_model import Lasso
lasso_model = Lasso(alpha=548)
lasso_model.fit(X_train_pca,y_train)

print('Coefficiet:',lasso_model.coef_,'\n','Intercept',lasso_model.intercept_)

lasso_coef = lasso_model.coef_

ypred_train = lasso_model.predict(X_train_pca)
ypred_test = lasso_model.predict(X_test_pca)

print("Train Score:", r2_score(y_train, ypred_train))
print('Test Score:', r2_score(y_test, ypred_test))


In [ ]:
d = dict()
for i in range(0,len(lasso_coef)):
    if lasso_coef[i] != 0:
        d[i] = lasso_coef[i]  

non_zero_indices = list(d.keys())

X_train_selected = X_train_pca[:, non_zero_indices]

X_test_selected = X_test_pca[:, non_zero_indices]

lasso_model_selected = Lasso(alpha=855)
lasso_model_selected.fit(X_train_selected, y_train)

ypred_train_selected = lasso_model_selected.predict(X_train_selected)
ypred_test_selected = lasso_model_selected.predict(X_test_selected)

train_score_selected = r2_score(y_train, ypred_train_selected)
test_score_selected = r2_score(y_test, ypred_test_selected)

print("Train Score (selected features):", train_score_selected)
print("Test Score (selected features):", test_score_selected)

In [ ]:
from sklearn.model_selection import GridSearchCV

estimator = Lasso()
param_grid = {"alpha":list(range(500,1000))}

grid_model = GridSearchCV(estimator,param_grid,cv = 5,scoring = 'r2')
grid_model.fit(X_train_selected,y_train)
grid_model.best_params_

#### 4. Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge
ridge_model = Ridge(alpha = 500)
ridge_model.fit(X_train_pca,y_train)

print('Coefficiet:',ridge_model.coef_,'\n','Intercept',ridge_model.intercept_)

ypred_train = ridge_model.predict(X_train_pca)
ypred_test = ridge_model.predict(X_test_pca)

print("Train Score:", r2_score(y_train, ypred_train))
print('Test Score:', r2_score(y_test, ypred_test))


In [ ]:
from sklearn.model_selection import GridSearchCV

estimator = Ridge()
param_grid = {"alpha":list(range(500,1000))}

grid_model = GridSearchCV(estimator,param_grid,cv = 5,scoring = 'r2')
grid_model.fit(X_train_pca,y_train)
grid_model.best_params_

#### 5. Elastic Net Regression

In [ ]:
from sklearn.linear_model import ElasticNet
elastic_model = ElasticNet(alpha = 5,l1_ratio = 0.9)
elastic_model.fit(X_train_pca,y_train)

print('Coefficiet:',elastic_model.coef_,'\n','Intercept',elastic_model.intercept_)

ypred_train = elastic_model.predict(X_train_pca)
ypred_test = elastic_model.predict(X_test_pca)

print("Train Score:", r2_score(y_train, ypred_train))
print('Test Score:', r2_score(y_test, ypred_test))


In [ ]:
from sklearn.model_selection import GridSearchCV

estimator = ElasticNet()

param_grid = {'alpha' :list(range(1,10)),'l1_ratio': [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1] }

grid_model = GridSearchCV(estimator,param_grid,cv = 5,scoring = 'r2')
grid_model.fit(X_train_pca,y_train)
grid_model.best_params_

#### Decesion Tree

In [ ]:
# 6 DecisionTree
# Modeling
from sklearn.tree import DecisionTreeRegressor
decision_model = DecisionTreeRegressor(random_state = 5)
decision_model.fit(X_train_pca,y_train)

# Prediction
ypred_train = decision_model.predict(X_train_pca)
ypred_test = decision_model.predict(X_test_pca)

# Accuracy and score
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
print("Train Accuracy: ",r2_score(y_train,ypred_train))
print("Test Accuracy: ",r2_score(y_test,ypred_test))
print('Cross Validation Score: ',cross_val_score(decision_model,X,y,cv = 5).mean())

In [ ]:
# 7. RandomForest
# Modelling
from sklearn.ensemble import RandomForestRegressor
random_model = RandomForestRegressor(random_state = 1)
random_model.fit(X_train_pca,y_train)

# predicting
ypred_train = random_model.predict(X_train_pca)
ypred_test = random_model.predict(X_test_pca)

# Accuracy Score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
print("Train accuracy = ",r2_score(y_train,ypred_train))
print("Test accuracy = ",r2_score(y_test,ypred_test))
print('Cross Validation Score:',cross_val_score(random_model,X,y,cv = 5).mean())

In [ ]:
## 8. GradientBoosting
# import the model
from sklearn.ensemble import GradientBoostingRegressor
gradient_model = GradientBoostingRegressor()
gradient_model.fit(X_train_pca,y_train)

# Prediction
ypred_train = gradient_model.predict(X_train_pca)
ypred_test = gradient_model.predict(X_test_pca)

#Evalaution
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
print('Train Score:',r2_score(y_train,ypred_train))
print('Test Score:',r2_score(y_test,ypred_test))
print('Cross Validation Score:',cross_val_score(gradient_model,X,y,cv =5).mean())

In [ ]:
## 9. XGBoosting
from xgboost import XGBRegressor
xgboost_model = XGBRegressor()
xgboost_model.fit(X_train_pca,y_train)

# prediction
ypred_train = xgboost_model.predict(X_train_pca) 
ypred_test = xgboost_model.predict(X_test_pca)

# Evoluation
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
print('Train Score:',r2_score(y_train,ypred_train))
print('Test Score:',r2_score(y_test,ypred_test))
print('Cross validation score:',cross_val_score(xgboost_model,X,y,cv = 5).mean())

**This the test data that which we want to predict**

In [ ]:
test_data = pd.read_csv('test.csv')
test_data

In [ ]:
test_data.info()

In [ ]:
ide = test_data['Id']

In [ ]:
dict_types = dict(test_data.dtypes)
dict_types
continous = []
count = []
categorical = []
for name,type in dict_types.items():
    if type == str('float64'):
        continous.append(name)
    elif type == str('int64'):
        count.append(name)
    else:
        categorical.append(name)
print('The Continous variables are:  ',continous)
print('\nThe Count variables are:  ',count)
print('\nThe Categorical variables are:  ',categorical)
print(len(continous),len(count),len(categorical))


In [ ]:
numerical_data = test_data.drop(categorical,axis =1)
numerical_data

In [ ]:
numerical_data.drop('Id',axis =1,inplace = True)
numerical_data.corr()

In [ ]:
test_data.drop('Fence',axis = 1 ,inplace = True)
test_data.drop('PoolQC',axis = 1 ,inplace = True)
test_data.drop('MiscFeature',axis = 1 ,inplace = True)
test_data.drop('Alley',axis = 1 ,inplace = True)
test_data.drop('Id',axis =1,inplace = True)

In [ ]:
test_data['LotFrontage'].fillna(test_data['LotFrontage'].mean(),inplace = True)
test_data['MasVnrArea'].fillna(test_data['MasVnrArea'].mean(),inplace = True)
test_data['GarageYrBlt'].fillna(test_data['GarageYrBlt'].mean(),inplace = True)

In [ ]:
test_data['BsmtQual'].fillna(test_data['BsmtQual'].mode()[0],inplace = True)    
test_data['BsmtCond'].fillna(test_data['BsmtCond'].mode()[0],inplace = True) 
test_data['BsmtExposure'].fillna(test_data['BsmtExposure'].mode()[0],inplace = True)
test_data['BsmtFinType1'].fillna(test_data['BsmtFinType1'].mode()[0],inplace = True)
test_data['BsmtFinType2'].fillna(test_data['BsmtFinType2'].mode()[0],inplace = True)
test_data['MasVnrType'].fillna(test_data['MasVnrType'].mode()[0],inplace = True)
test_data['GarageType'].fillna(test_data['GarageType'].mode()[0],inplace = True)
test_data['GarageFinish'].fillna(test_data['GarageFinish'].mode()[0],inplace = True)       
test_data['GarageQual'].fillna(test_data['GarageQual'].mode()[0],inplace = True)       
test_data['GarageCond'].fillna(test_data['GarageCond'].mode()[0],inplace = True)
test_data['Electrical'].fillna(test_data['Electrical'].mode()[0],inplace = True)
test_data['Utilities'].fillna(test_data['Utilities'].mode()[0],inplace = True)
test_data['MSZoning'].fillna(test_data['MSZoning'].mode()[0],inplace = True)
test_data['SaleType'].fillna(test_data['SaleType'].mode()[0],inplace = True)
test_data['Exterior1st'].fillna(test_data['Exterior1st'].mode()[0],inplace = True)
test_data['Exterior2nd'].fillna(test_data['Exterior2nd'].mode()[0],inplace = True)
test_data['BsmtFinSF1'].fillna(test_data['BsmtFinSF1'].mode()[0],inplace = True)
test_data['BsmtFinSF2'].fillna(test_data['BsmtFinSF2'].mode()[0],inplace = True)
test_data['BsmtUnfSF'].fillna(test_data['BsmtUnfSF'].mode()[0],inplace = True)
test_data['TotalBsmtSF'].fillna(test_data['TotalBsmtSF'].mode()[0],inplace = True)
test_data['BsmtFullBath'].fillna(test_data['BsmtFullBath'].mode()[0],inplace = True)
test_data['BsmtHalfBath'].fillna(test_data['BsmtHalfBath'].mode()[0],inplace = True)
test_data['KitchenQual'].fillna(test_data['KitchenQual'].mode()[0],inplace = True)
test_data['Functional'].fillna(test_data['Functional'].mode()[0],inplace = True)
test_data['GarageCars'].fillna(test_data['GarageCars'].mode()[0],inplace = True)
test_data['GarageArea'].fillna(test_data['GarageArea'].mode()[0],inplace = True)
test_data['FireplaceQu'].fillna(test_data['FireplaceQu'].mode()[0],inplace = True)

In [ ]:
dict_types = dict(test_data.dtypes)
dict_types
continous = []
count = []
categorical = []
for name,type in dict_types.items():
    if type == str('float64'):
        continous.append(name)
    elif type == str('int64'):
        count.append(name)
    else:
        categorical.append(name)
print('The Continous variables are:  ',continous)
print('\nThe Count variables are:  ',count)
print('\nThe Categorical variables are:  ',categorical)
print(len(continous),len(count),len(categorical))

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(drop='first')
cat = enc.fit_transform(test_data[categorical])
test_data.drop(categorical,axis=1,inplace = True)
cat_df = pd.DataFrame(cat.toarray(), columns=enc.get_feature_names_out(categorical))
test_data = pd.concat([test_data, cat_df], axis='columns')

# In the test some of the categorical data is missing that data is present is train data but not there in test data.
# So it is problem to transform the data so that is reason we have to identify them and put '0'. if we zero if it is
# present in train but not in test we can keep them without any problem
missing_features = [
    'Utilities_NoSeWa',
    'Condition2_RRAe',
    'Condition2_RRAn',
    'Condition2_RRNn',
    'HouseStyle_2.5Fin',
    'RoofMatl_CompShg',
    'RoofMatl_Membran',
    'RoofMatl_Metal',
    'RoofMatl_Roll',
    'Exterior1st_ImStucc',
    'Exterior1st_Stone',
    'Exterior2nd_Other',
    'Heating_GasA',
    'Heating_OthW',
    'Electrical_Mix',
    'GarageQual_Fa'
]


num_rows = 1459

# Create a DataFrame with all missing features set to 0
missing_features_df = pd.DataFrame(np.zeros((num_rows, len(missing_features))), columns=missing_features)

# Concatenate the missing features DataFrame with the existing test_data
test1_data = pd.concat([test_data, missing_features_df], axis=1)
test1_data
final_feature_list = list(data.columns)

# Reorder columns to match final feature list
test1_data = test1_data.reindex(columns=final_feature_list)
test1_data.drop('SalePrice',axis=1,inplace = True)

In [ ]:
X = test1_data
X

In [ ]:
X_test = sc_model.transform(X)

In [ ]:
X_test_pca = pca_model.transform(X_test)

In [ ]:
X_test_pca.shape

In [ ]:
# Assuming testpred contains the predicted values

# The Final Model is Ridge Regression
testpred = ridge_model.predict(X_test_pca)

# Convert the predicted values to a DataFrame with IDE numbers as the index
pred_df = pd.DataFrame(testpred, columns=['SalePrice'], index=ide)

# Save the DataFrame to a CSV file
pred_df.to_csv('predicted_sale_price.csv')

In [ ]:
data_stored = pd.read_csv('predicted_sale_price.csv')
data_stored